In [38]:
from keras.datasets import mnist
from tensorflow import keras
from keras import models 
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical

(X, y), (X_test, y_test) = mnist.load_data()

In [46]:
X_train = X.reshape((60000, 28*28))
X_train = X_train/255

X_test = X_test.reshape((10000, 28*28))
X_test = X_test/255

y_train=to_categorical(y)
y_test=to_categorical(y_test)

X_train.shape,y_train.shape,X_test.shape,y_test.shape, X.shape, y.shape

((60000, 784),
 (60000, 10),
 (10000, 784),
 (10000, 10),
 (60000, 28, 28),
 (60000,))

# softmax

In [210]:
net = models.Sequential()
net.add(layers.Dense(10, activation='softmax', input_shape=(28*28,)))
net.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

In [146]:
net.fit(X_train, y_train, epochs=20)

Epoch 1/20
1875/1875 [==============================] - 2s 939us/step - loss: 0.7727 - accuracy: 0.8185
Epoch 2/20
1875/1875 [==============================] - 2s 914us/step - loss: 0.4557 - accuracy: 0.8806
Epoch 3/20
1875/1875 [==============================] - 2s 950us/step - loss: 0.4032 - accuracy: 0.8908
Epoch 4/20
1875/1875 [==============================] - 2s 906us/step - loss: 0.3766 - accuracy: 0.8971
Epoch 5/20
1875/1875 [==============================] - 2s 870us/step - loss: 0.3600 - accuracy: 0.9003
Epoch 6/20
1875/1875 [==============================] - 2s 933us/step - loss: 0.3481 - accuracy: 0.9036
Epoch 7/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3393 - accuracy: 0.9057
Epoch 8/20
1875/1875 [==============================] - 2s 804us/step - loss: 0.3320 - accuracy: 0.9075
Epoch 9/20
1875/1875 [==============================] - 2s 844us/step - loss: 0.3263 - accuracy: 0.9088
Epoch 10/20
1875/1875 [==============================] - 1s 723us/

In [148]:
net.evaluate(X_test,y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.2877 - accuracy: 0.9200


[0.287660151720047, 0.9200000166893005]

softmax regression 결과 정확도는 92%이다

# SVM

In [6]:
from sklearn import svm

In [7]:
clf=svm.SVC(C=200,kernel='rbf',gamma=0.01,cache_size=8000,probability=False)

In [8]:
clf.fit(X_train,y)

SVC(C=200, break_ties=False, cache_size=8000, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

데이터 shape가 달라 연산이 안돼서 다른 방법으로 접근

In [24]:
from sklearn.datasets import fetch_openml

mnist=fetch_openml('mnist_784',version=1)
print(list(mnist))  
svmX,svmy=mnist['data'],mnist['target']

['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url']


In [25]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

svmX_train,svmX_test,svmy_train,svmy_test=train_test_split(svmX,svmy,stratify=svmy,test_size=0.3,random_state=42)

In [27]:
from sklearn.svm import LinearSVC

svm=LinearSVC(random_state=42)
svm.fit(svmX_train,svmy_train)

C:\Users\yunju\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [29]:
from sklearn.metrics import accuracy_score
y_pred=svm.predict(svmX_train)
accuracy_score(svmy_train,y_pred)

0.8882040816326531

svm의 정확도는 88%이다

#  randomforest

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [179]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [181]:
cross_val_score(rf,X_train,y_train,cv=3,scoring="accuracy")

array([0.88385, 0.87965, 0.88585])

In [182]:
a=[0.88385, 0.87965, 0.88585]
np.mean(a)

0.8831166666666667

randomforest는 정확도 88% 정도이다

softmax는 92%, SVM과 randomforest는 88%의 정확도를 가진다.
softmax는 50번 연산을 해도 눈에 띄는 정확도 개선이 보이지 않아서 따로 20번만 실행했다.
randomforest는 softmax에서 epochs를 늘리는 것 처럼 tree를 늘리면 조금씩 정확도가 상승할 것 같지만 overfitting을 구분할 수 없어서 default값으로 실행했다.
SVM은 gamma값을 조정하면 정확도를 더 올릴 수 있는 것 같은데, 어느 지점이 overfitting인지도 모르고, 어느정도가 적정값인지 몰라 default값으로 실행하였다. softmax, randomforest에서 썼던 데이터의 shape로는 계산할 수 없어 새로운 데이터 shape로 SVM을 했다.